## 说明

该方法与常见基于北向资金的方法不同，它将价格因素纳入考虑

当然期望能比普通的基于北向的方法好一点，但不能确定，尤其是面对目前的市场，北向卖飞的比比皆是

基于北向的策略在17年以后进行回测天生就有未来数据，因为北向资金塑造A股是目前的共识，所以用北向的数据就等于是用“未来知识”进行测试，高于大盘走势很正常。

在市场风格已经被北向的偏好“极端化”的现在，未来跟随北向的策略是否有效很不确定

本策略将以普通的北向跟踪策略作为比较基准




### 共有4种情况

|     | volume-up  | volume-down |
|  ----  | ----  | ---- |
| price-up  | 1.常见情况 | 2.瞎炒or港资卖飞 |
| price-down  | 3.错杀or港资瞎买 | 4.被抛弃  |

将目标股票设置为<b>当前持仓成本-现价 </b>尽可能大

所以最优先的是情况3，然后是<b>量价齐升</b>，即情况1，再次是情况2，要尽可能避免情况4

### 难点
- 对复权的处理，

In [ ]:

## No.2 
'''
    volume_roll 过去一年北向资金成交额数据
    price_roll  过去一年收盘价
    
'''

# get_price(security, start_date='2015-01-01', end_date='2015-12-31', frequency='daily', 
#           fields=['avg'], skip_paused=False, fq='pre', panel=False)
# q = query()

# 初始持仓量，结束持仓量，初始持仓量调整，每日北向成交额

import numpy as np
from jqdata import finance

# 获取过去一年，每日北向持仓数据
# output:df1：code是col[0]，col为date
def get_vol_roll(end_date, n_days, securities):
    
    df1 = pd.DataFrame()
    for ii in range(n_days):
        curr_date = (end_date + datetime.timedelta(-ii)).strftime("%Y-%m-%d")
        
        q1 = query(finance.STK_HK_HOLD_INFO.code, finance.STK_HK_HOLD_INFO.share_number
                  ).filter(finance.STK_HK_HOLD_INFO.date == curr_date, finance.STK_HK_HOLD_INFO.code.in_(securities)
                          ).limit(4000)
        df = finance.run_query(q1)
        
        df.rename(columns={'share_number':curr_date}, inplace = True)
        
        df1 = df1.join(df, on = 'code')
    
    return df1

def get_price_roll(end_date, n_days, securities):
    
    df2 = pd.DataFrame()
    for ii in range(n_days):
        curr_date = (end_date + datetime.timedelta(-ii)).strftime("%Y-%m-%d")  
        
        df = get_price(securities, start_date=curr_date, end_date=curr_date, frequency='daily', 
              fields=['avg'], skip_paused=False, fq=None, panel=False)
        
        # df4.rename(columns = {'avg':'start_close'}, inplace = True)
        assert(df['time'].drop_duplicates().shape[0] == 1)
        assert(df['time'].drop_duplicates()[0].strftime("%Y-%m-%d") == curr_date)
        
        df = df.drop('time',1)
        
        df.rename(columns={'avg':curr_date}, inplace = True)
        
        df2 = df2.join(df, on = 'code')
        
    return df2

def get_volumn_price(date=None, pre = 1, securities = None)
    
    
def cal_roll_cost()
    

def correct_hold(code_hold, date):
    
    # 起始日至送转日期间送股，转股生成的扩展倍数
    # 如果直接使用不复权数据，那这个也没意义了
    
    q = query(finance.STK_XR_XD.code, finance.STK_XR_XD.devidend_ratio, finance.STK_XR_XD.transfer_ratio
              ).filter(finance.STK_XR_XD.code == code_hold.code, finance.STK_XR_XD.change_date > date).order_by(finance.STK_XR_XD.report_date).limit(1000)
    df = finance.run_query(q)

    expand = 1
    for row in df.iterrows():
        if row['devidend_ratio'] and row['transfer_ratio']:
            expand *= (1 + devidend_ratio + transfer_ratio)
        else:
            expand *= (1 + (row['devidend_ratio'] if row['dividend_ratio'] is not None else 0) + (
                row['transfer_ratio'] if row['transfer_ratio'] is not None else 0))
            
    return(code_hold[-1] * expand)


def get_hold_price(date=None, pre = 1, securities = None):
    
    if date == None:
        date = datetime.datetime.now()
    start_date = date + datetime.delta(-365)
    
    dates = []
    for ii in range(365):
        dates.append((date + date.delta(-ii)).strftime("%Y-%m-%d"))
    
    # 获得结束日的北向持仓股票列表
    if securities == None:
        q0 = query(finance.STK_HK_HOLD_INFO.code).filter(finance.STK_HK_HOLD_INFO.date == date)
        df0 = finance.run_query(q0)
        
        securities = np.array(df0['code']).flatten.tolist()
    
    df1 = get_vol_roll(end_date = date, n_days = 366, securities = securities)
    print("shape of df1: ", df1.shape)
    
    # 获取开始日至收盘日之间所有日期每日的平均交易价格，注意选用不复权价格，以与每日持有量统一
    df2 = get_price_roll(end_date, n_days, securities)
    print("shape of df2: ", df2.shape)
    
    # 注意需要df3形式尽量与df2一致
    df3 = hold_to_buis(df1)
    
    cal(df2, df3)
    
    return [df1, df2]



# 感觉还是要引入复权因素的影响
def get_result(df_hold, df_price):
    
    assert df_hold.shape == df_price.shape
    result = df_hold.code
    
    # 追溯调整起始持仓数量，这里仍然不够精确，因为如果在送转之前卖出，那么会高估起始持仓数
    # hold_corred = correct_hold(df_hold.iloc[:,[0,1]])
    hold_corred = df_hold.iloc[:, [0,1]].apply(correct_hold, axis = 1, args = (df_hold.columns[1]))
    result['hold_change'] = df_hold.iloc[:,-1] - hold_corred
    
    # 计算期间的资金
    # 这里应该是计算每日成交额，正常应该继续调整除权除息日的持股数量，以保证当日成交股数的正确
    # 调整除权除息日初始股数
    
    # 循环相减，获得每一天的差额
    df.change = 
    
    # 计算每只股票的资金流入净额  
    df_cash = df_change.iloc[:,1:] * df_price.iloc[:,2:]
    result['df_cash_sum'] = df_cash.apply(sum, axis = 1)
    
    # 
    result['avg_buy'] = result['df_cash_sum'] / result['hold_change']
    
def main():
    
    get_result(get_hold_price())
    
    
    # 起始日各股票HK持仓量
#     q2 = query(finance.STK_HK_HOLD_INFO.code, finance.STK_HK_HOLD_INFO.share_number
#                   ).filter(finance.STK_HK_HOLD_INFO.date == start_date, finance.STK_HK_HOLD_INFO.code.in_(securities)).limit(10000)
#     df2 = finance.run(q2)
#     df2.rename(columns={'share_number':'start_share_n'}, inplace = True)
    
#     df = df.join(df2, on = 'code')
    
   
    
#     df4 = get_price(securities, start_date=start_date, end_date=start_date, frequency='daily', 
#           fields=['close'], skip_paused=False, fq='pre', panel=False)
#     df4.rename(columns = {'close':'start_close'}, inplace = True)
#     df = df.join(df4, on = 'code')
    
#     df5 = get_price(securities, start_date= date, end_date= date, frequency='daily', 
#           fields=['close'], skip_paused=False, fq='pre', panel=False)
#     df5.rename(columns = {'close':'end_close'}, inplace = True)
#     df = df.join(df5, on = 'code')

    
   
    


    # dividend_ratio  每10股送XX股     transfer_ratio  每10股转增 XX股   dividend_number   transfer_number

# 返回最近一年操作最终的平均持仓成本
'''
    干扰因素：拆股，全都采用前复权价格，计算中直接用总金额计算，规避干扰因素
    使用前复权可以解决分红问题，但是拆股送股，由于初始的股数并未追溯调整，所以要追溯调整股数使其准确，注意要调整的是股票数的差额
    即将起始股数调多，然后计算差额
    A股股票现金分红少见，而送转多见，所以这个调整是必须的
'''
'''
    想来想去，好像每天的成交价和成交量都不重要，成交额就可以都代表了，只需要首尾的持仓量即可
    所以是牺牲复用换取单次计算的简便
'''
# paras：成交额，价格
def cost_curr(volume_roll, price_roll):
    # （买入投的钱 - 卖出拿到的钱） / （终止日持股数 - 起始日持股数）
    #  
    return x





finance.run_query(query(finance.STK_CAPITAL_CHANGE).filter(finance.STK_CAPITAL_CHANGE.code==code).limit(n))
# code, change_date, change_reason_id, change_reason, share_total, share_convert, 


#
# 期望是能得到每只股票每天的净买额数据，但聚宽好像没有这个数据，之后看看其它框架有没有这个数据，没有的话只能取每天的成交数据和成交均价自己计算了
import

In [2]:
import pandas as pd

a = np.array([[1, 2, 3],
             [4, 5, 6]])
b = np.array([[1,2],
              [3,4],
              [5,6]])
a = pd.DataFrame(a)
b = pd.DataFrame(b)

a * a

,0,1,2
0,1,4,9
1,16,25,36


In [13]:
df4['time'].drop_duplicates()[0].strftime("%Y-%m-%d")

'2021-07-09'

In [3]:
df4 = get_price(['000001.XSHE','000002.XSHE'], start_date='2021-7-9', end_date='2021-7-9', frequency='daily', 
          fields=['avg'], skip_paused=False, fq=None, panel=False)

In [23]:
dates2 = []
for everyday in dates:
    dates2.append(everyday.strftime("%Y-%m-%d"))
dates2

['2021-07-27', '2021-07-26', '2021-07-25', '2021-07-24', '2021-07-23']

In [18]:
df4.columns

Index(['time', 'code', 'avg'], dtype='object')

In [26]:
from jqdata import finance

q1 = query(finance.STK_HK_HOLD_INFO.code, finance.STK_HK_HOLD_INFO.day, finance.STK_HK_HOLD_INFO.share_number
                  ).filter(finance.STK_HK_HOLD_INFO.day.in_(dates2)).limit(10000)
df = finance.run_query(q1)

In [27]:
df

,code,day,share_number
0,000001.XSHE,2021-07-23,2210256695
1,000002.XSHE,2021-07-23,617963350
2,000005.XSHE,2021-07-23,2010
3,000006.XSHE,2021-07-23,5512403
4,000008.XSHE,2021-07-23,40516319
5,000009.XSHE,2021-07-23,74487723
6,00001.XHKG,2021-07-23,24937579
7,000011.XSHE,2021-07-23,2312219
8,000012.XSHE,2021-07-23,152888617
9,000016.XSHE,2021-07-23,9062577


In [10]:
df['code'].drop_duplicates().shape

(2960,)

In [28]:
get_query_count()

NameError: name 'get_query_count' is not defined

In [ ]:
## code in trash


       
    
from jqdata import finance
q=query(finance.STK_CAPITAL_CHANGE.change_date, finance.STK_CAPITAL_CHANGE.change_reason, finance.STK_CAPITAL_CHANGE.share_total, finance.STK_CAPITAL_CHANGE.share_convert).filter(finance.STK_CAPITAL_CHANGE.code=='600276.XSHG',finance.STK_CAPITAL_CHANGE.pub_date>'2015-01-01').limit(10)
df=finance.run_query(q)
print(df)


# 获取所有股票期间HK交易信息
from jqdata import finance
qq = query(finance.STK_HK_HOLD_INFO).filter(finance.STK_HK_HOLD_INFO.link_id = )
# code,name,share_number,day
df = finance.run_query

In [ ]:


## 目前的代码里面，忽略了配股的影响。另外，对送转采取的策略是假设